<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/afdesign_hotspot_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AfDesign binder + hotspot test

In [1]:
#@title
%%bash
if [ ! -d af_backprop ]; then
  git clone https://github.com/sokrypton/af_backprop.git
  pip -q install biopython dm-haiku==0.0.5 ml-collections py3Dmol
fi
if [ ! -d params ]; then
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabDesign/main/af/design.py

Cloning into 'af_backprop'...


In [2]:
#@title import libraries
import sys
sys.path.append('/content/af_backprop')

import os
from google.colab import files
import numpy as np
from IPython.display import HTML
from design import mk_design_model, clear_mem

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

In [3]:
%%bash
wget -qnc https://raw.githubusercontent.com/casperg92/MaSIF_colab/main/example/monomerexample.pdb

In [ ]:
clear_mem()
model = mk_design_model(protocol="binder")
model.prep_inputs(pdb_filename="monomerexample.pdb", chain="A",
                  binder_len=18,
                  hotspot="33,17,6,34,30,19,37,15")

print("target_length",model._target_len)
print("binder_length",model._binder_len)
print("weights",model.opt["weights"])

In [10]:
model.restart()
model.opt["con_cutoff"] = 14.0
model.opt["weights"].update({'msa_ent': 0.0, 'plddt': 0.0, 'pae_intra': 0.0, 'con_intra': 0.0,
                             'pae_inter': 1.0, 'con_inter': 0.5})
model.design_3stage(soft_iters=100, temp_iters=100, hard_iters=10)

1	model: [1] recycles: 0 soft: 0.000 temp: 0.500 loss: 3.874 pae_intra: 0.204 pae_inter: 0.891 plddt: 0.358 con_intra: 2.697 con_inter: 5.966
2	model: [1] recycles: 0 soft: 0.010 temp: 0.500 loss: 2.626 pae_intra: 0.219 pae_inter: 0.804 plddt: 0.391 con_intra: 2.715 con_inter: 3.644
3	model: [1] recycles: 0 soft: 0.020 temp: 0.500 loss: 2.174 pae_intra: 0.224 pae_inter: 0.774 plddt: 0.450 con_intra: 2.682 con_inter: 2.799
4	model: [0] recycles: 0 soft: 0.030 temp: 0.500 loss: 1.708 pae_intra: 0.264 pae_inter: 0.648 plddt: 0.534 con_intra: 1.842 con_inter: 2.120
5	model: [1] recycles: 0 soft: 0.040 temp: 0.500 loss: 1.980 pae_intra: 0.240 pae_inter: 0.718 plddt: 0.487 con_intra: 1.980 con_inter: 2.524
6	model: [1] recycles: 0 soft: 0.051 temp: 0.500 loss: 1.570 pae_intra: 0.246 pae_inter: 0.674 plddt: 0.517 con_intra: 1.724 con_inter: 1.792
7	model: [0] recycles: 0 soft: 0.061 temp: 0.500 loss: 1.420 pae_intra: 0.258 pae_inter: 0.598 plddt: 0.520 con_intra: 1.529 con_inter: 1.644
8	mode

In [11]:
HTML(model.animate())

In [12]:
model.get_seqs()

['DDLQQFEQNMPPALQQAM']

In [13]:
model.plot_pdb()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
model.save_pdb(f"{model.protocol}.pdb")